In [25]:
dftrain = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
dftest  = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")
ytrain =  dftrain.pop('survived')
ytest = dftest.pop('survived')
dftrain.head()
print(ytest.head)

<bound method NDFrame.head of 0      0
1      0
2      1
3      1
4      1
      ..
259    1
260    0
261    0
262    0
263    1
Name: survived, Length: 264, dtype: int64>


In [9]:
CATEGORICAL_COL = ['sex','n_siblings_spouses', 	'parch', 	'class', 	'deck', 	'embark_town', 	'alone']
NUMERIC_COL = ['age', 'fare']

feature_col = []

for feature_name in CATEGORICAL_COL:
  vocabulary = dftrain[feature_name].unique()
  feature_col.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name,vocabulary_list=vocabulary))

for feature_name in NUMERIC_COL:
  feature_col.append(tf.feature_column.numeric_column(feature_name,dtype=float))  

print(feature_col)

[VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='n_siblings_spouses', vocabulary_list=(1, 0, 3, 4, 2, 5, 8), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='parch', vocabulary_list=(0, 1, 2, 5, 3, 4), dtype=tf.int64, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='class', vocabulary_list=('Third', 'First', 'Second'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='deck', vocabulary_list=('unknown', 'C', 'G', 'A', 'B', 'D', 'F', 'E'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Southampton', 'Cherbourg', 'Queenstown', 'unknown'), dtype=tf.string, default_value=-1, num_oov_buckets=0), VocabularyListCategoricalColumn(key='alone', vocabulary_list=('n', 'y'), dtype=tf.string, def

In [11]:
def make_input_fn(data_df, label_df, num_epochs = 10, shuffle = True, batch_size = 32):
  def input_function(): #inner function, this will be returned
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df),label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds #return the dataset
  return input_function #return a function object for use

# preparing data to be fed to the model

In [ ]:
train_input_fn = make_input_fn(dftrain, ytrain)
test_input_fn = make_input_fn(dftest, ytest, num_epochs = 1, shuffle = False)

In [24]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_col) #select the model to train

linear_est.train(train_input_fn) #train the model
result = linear_est.evaluate(test_input_fn) #store the result of the testing model with dftest and ytest data

clear_output()
print(result['accuracy'])
print(result)
# this accurancy is how close the model's value for ylabels are  to the ytest data

{'accuracy': 0.7689394, 'accuracy_baseline': 0.625, 'auc': 0.82923174, 'auc_precision_recall': 0.77534246, 'average_loss': 0.51933944, 'label/mean': 0.375, 'loss': 0.51095426, 'precision': 0.7567568, 'prediction/mean': 0.27876875, 'recall': 0.56565654, 'global_step': 200}


In [33]:
result_predict = list(linear_est.predict(test_input_fn))
print(dftest.loc[2])
print(ytest.loc[2])
print(result_predict[2]['probabilities'][1])

INFO:tensorflow:Calling model_fn.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/engine/base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp9zqekw69/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
sex                        female
age                            58
n_siblings_spouses              0
parch                           0
fare                        26.55
class                       First
deck                            C
embark_town           Southampton
alone                           y
Name: 2, dtype: object
1
0.4702224
